In [1]:
import pandas as pd
import numpy as np

pd.set_option("mode.copy_on_write", True)

import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.linear_model import TweedieRegressor
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv("../data/InsNova_data_2023_train.csv")
df.head()

,id,veh_value,exposure,veh_body,veh_age,gender,area,agecat,engine_type,max_power,...,marital_status,e_bill,time_of_week_driven,time_driven,trm_len,credit_score,high_education_ind,clm,numclaims,claimcst0
0,1,0.77,0.444504,SEDAN,4,M,D,3,petrol,147,...,S,1,weekday,6pm - 12am,6,640.448137,1.0,0,0,0.0
1,2,4.45,0.562183,STNWG,1,M,A,3,petrol,158,...,S,1,weekday,6am - 12pm,12,683.749691,0.0,0,0,0.0
2,3,4.90,0.465244,STNWG,1,F,A,3,petrol,159,...,M,1,weekday,6pm - 12am,6,653.656117,1.0,0,0,0.0
3,4,0.48,0.271039,PANVN,4,M,A,4,petrol,80,...,S,1,weekday,12pm - 6pm,12,642.574671,0.0,0,0,0.0
4,5,0.85,0.141624,SEDAN,4,F,A,5,petrol,126,...,S,0,weekday,6am - 12pm,6,647.175035,0.0,0,0,0.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22619 entries, 0 to 22618
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     22619 non-null  int64  
 1   veh_value              22619 non-null  float64
 2   exposure               22619 non-null  float64
 3   veh_body               22619 non-null  object 
 4   veh_age                22619 non-null  int64  
 5   gender                 22619 non-null  object 
 6   area                   22619 non-null  object 
 7   agecat                 22619 non-null  int64  
 8   engine_type            22619 non-null  object 
 9   max_power              22619 non-null  int64  
 10  driving_history_score  22619 non-null  float64
 11  veh_color              22619 non-null  object 
 12  marital_status         22619 non-null  object 
 13  e_bill                 22619 non-null  int64  
 14  time_of_week_driven    22619 non-null  object 
 15  ti

In [4]:
df["high_education_ind"] = df["high_education_ind"].astype("object")

In [5]:
# Identify categorical columns
categorical_cols = df.select_dtypes(include=["object"]).columns

In [6]:
# Perform one-hot encoding for categorical variables
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

In [7]:
# Separate predictors and target
X = df.drop(columns=["id", "clm", "numclaims", "claimcst0"], axis=1)
y = df["claimcst0"]

In [8]:
clf = linear_model.TweedieRegressor(
    power=1.25,
    alpha=0,
    link="log",
    solver="newton-cholesky",
    max_iter=10000,
    warm_start=True,
)
clf.fit(X, y)
clf.score(X, y)

0.041294767918974795

## Test Prediction

In [9]:
test = pd.read_csv("../data/InsNova_data_2023_vh.csv")

In [10]:
test["high_education_ind"] = test["high_education_ind"].astype("object")

In [11]:
# Identify categorical columns
categorical_cols = test.select_dtypes(include=["object"]).columns
# Perform one-hot encoding for categorical variables
test = pd.get_dummies(test, columns=categorical_cols, drop_first=True)
test = test.rename(columns={"high_education_ind_1": "high_education_ind_1.0"})
# Separate predictors and target
X_test = test.drop(columns=["id"], axis=1)

In [12]:
y_pred = clf.predict(X_test)

In [13]:
test["Predict"] = y_pred

In [14]:
test["Predict"].describe()

count    22620.000000
mean       162.986269
std         88.773806
min          0.000002
25%         99.009049
50%        142.799066
75%        205.424140
max        994.627499
Name: Predict, dtype: float64

In [15]:
submission = test[["id", "Predict"]]
submission

,id,Predict
0,1,109.880660
1,2,82.864474
2,3,52.791959
3,4,172.807065
4,5,74.284071
...,...,...
22615,22616,133.726305
22616,22617,136.862591
22617,22618,251.799831
22618,22619,253.447642


In [16]:
submission.to_csv("../output/Tweedie_submission.csv", index=False)